In [29]:
#빅카인즈 뉴스 정보 수집하기

#Step 1. 필요한 모듈을 로딩합니다
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys       
import numpy           
import pandas  as pd    
import math 
import os
import random
import pyautogui

#Step 2. 필요한 정보를 수집합니다
query_txt_1 = input('1.크롤링할 뉴스의 키워드는 무엇입니까?: ')
query_txt = query_txt_1.replace('"','')
poham_keyword = input('2.반드시 포함해야 할 단어가 있다면 적어주세요(여러 단어일 경우 , 로 구분해 주세요): ')
no_keyword = input('3.제외할 단어가 있다면 적어주세요:' )
start_date = input('4.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :')
end_date = input('5.조회를 종료할 날짜를 입력하세요(예:2017-12-31): ')
f_dir = input("6.파일을 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir="c:\\temp\\"

print("\n")

print("요청하신 데이터를 수집 중입니다")
print("\n")
print("잠시만 기다려 주세요~~~~~^^")
print("\n")

#Step 3.저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+'Bigkinds'+'-'+s+'-'+query_txt)
os.chdir(f_dir+'Bigkinds'+'-'+s+'-'+query_txt)

# Step 4. 크롬 드라이버 설정 
chrome_path = "c:/temp/chromedriver_85/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

# Step 5. 빅카인즈 사이트에 접속합니다
driver.get('https://www.bigkinds.or.kr/')
driver.maximize_window( )
print("Step 1.빅카인즈 홈페이지에 자동 접속 합니다. 잠시만 기다려 주세요 ^^")
time.sleep(random.randrange(5,10))

s_time = time.time( )     

# 팝업창이 있을 경우 닫기
try :
    driver.find_element_by_xpath(''' //*[@id="contents"]/div[3]/div/div[2]/div/div[2]/button''').click()
except :
    print('')
print('Step 2.날짜 조건과 검색어를 입력하여 요청하신 뉴스를 검색중입니다. 잠시 후 총 뉴스의 건수가 표시될 예정입니다~^^')

#Step 6. 검색 조건 입력하기
# 시작 날짜 입력하기
driver.find_element_by_id('date-filter-btn').click()
s_date = driver.find_element_by_xpath("""//*[@id="search-begin-date"]""")   
time.sleep(1)
s_date.click()
pyautogui.press('end')
for a in range(0,10):
    pyautogui.press('backspace')
for b in start_date :
    s_date.send_keys(b)
    time.sleep(0.3)
s_date.send_keys("\n")
    
# 종료 날짜 입력하기    
e_date = driver.find_element_by_xpath("""//*[@id="search-end-date"]""")
time.sleep(1)
driver.find_element_by_xpath("""//*[@id="search-end-date"]""").click()
e_date.click()
pyautogui.press('end')
for a in range(0,10):
    pyautogui.press('backspace')
for b in end_date :
    e_date.send_keys(b)
    time.sleep(0.3)

# 상세 검색 버튼 누르기
driver.find_element_by_id('detail-filter-btn').click()

# 반드시 포함하는 단어
orkeyword = driver.find_element_by_id('orKeyword1')
orkeyword.click()
orkeyword.clear()
orkeyword.send_keys(poham_keyword)
time.sleep(1)

# 제외할 단어
notkeyword = driver.find_element_by_id('notKeyword1')  
notkeyword.click()
notkeyword.clear()
notkeyword.send_keys(no_keyword)
time.sleep(1)

# 검색어 입력한 후 검색하기    
element = driver.find_element_by_id("total-search-key")
driver.find_element_by_id("total-search-key").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(1)

# Step 7. 크롤링 건수 추출 후 건수 입력받기
time.sleep(0.2)
driver.find_element_by_xpath("""//*[@id="collapse-step-2"]/div/div/div[2]/div[2]/div/div[2]/div[2]/select""").click()
driver.find_element_by_xpath("""//*[@id="collapse-step-2"]/div/div/div[2]/div[2]/div/div[2]/div[2]/select/option[4]""").click()

time.sleep(random.randrange(5,16))
total2 = driver.find_element_by_xpath("""//*[@id="total-news-cnt"]""").text
total = total2.replace(",","")
print("Step 3.검색된 기사의 개수는 총 %s 건 입니다"%total)
cnt = int(input('검색된 기사 중에서 크롤링 할 건수를 입력해주세요(최대:%s 건): ' %total) )
page_cnt = math.ceil(cnt / 100) 
print("\n")

company2=[ ]     # 언론사명 컬럼
title2=[ ]       # 게시글 제목 컬럼
date2=[ ]        # 게시글 날짜 컬럼
content2=[ ]     # 기사 본문 컬럼

no = 1           # 전체 게시글 번호

def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    #driver.execute_script("window.scrollBy(0,9300);")
    time.sleep(1)
        
# 페이지 번호 변경하는 for 반복문
for a in range(1,page_cnt+1) :
    news_title = 1   # 뉴스 제목 순서용 번호
    
    # 한 페이지에서 뉴스 글을 변경하는 for 반복문
    for b in range(0,100) :
        
        print('%s 번째 기사 정보를 수집합니다=========================================================' %no)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        n_list = soup.find('div','tab-content news-result-tab-content').find('div','tab-pane active')
        news_list = n_list.find_all('div','news-item__body')
        
        # 1. 언론사 정보 추출
        try :
            company = news_list[b].find('div','news-item__meta').find('a').get_text().split('\n')
        except :
            company = news_list[b].find('div','news-item__meta').get_text().split('\n')          
        company1 = company[1].strip()
        company2.append(company1)
        print('1.언론사명: ' , company1)
        
        
        # 2. 기사 제목 추출
        title = news_list[b].find('h4','news-item__title news-detail').get_text().replace('\n','').strip()
        title2.append(title)
        print('2.뉴스제목:' , title)
        
        # 3. 뉴스 보도 날짜
        date1 = news_list[b].find('div','news-item__meta').find('span','news-item__date')
        date = date1.get_text().strip().replace('-','').replace('>','')

        if date == '' or len(date) != 10:
                date = date1[3].get_text().strip().replace("\n","")

        date2.append(date)
        print("3.게시날짜 : %s" %date)
        
        # 4. 뉴스 본문
        driver.find_element_by_xpath('//*[@id="news-results"]/div[%s]/div[2]/h4' %news_title).click()
        news_title += 1
        time.sleep(2)
        
        html2 = driver.page_source
        soup2 = BeautifulSoup(html2, 'html.parser')
        
        cont=soup2.find("div","news-detail__content").get_text().replace("\n","").strip()
        if len(cont) == 0 :
            cont = '내용이 없는 기사입니다'

        content2.append(cont)
        print("4.뉴스내용 : %s" %cont)
        
        # 본문 기사 아래의 닫기 버튼 누르기
        driver.find_element_by_xpath('//*[@id="news-detail-modal"]/div/div/div[3]/button').click()
                       
        if no == cnt :
            break
        no += 1
        
        print('\n')
        
    scroll_down(driver)
        
    a += 1
    time.sleep(2)
    
    try :
        driver.find_element_by_link_text('%s' %a).click()
    except :
        driver.find_element_by_link_text('다음').click()

    # 다음 페이지 누르고 기다리기
    time.sleep(random.randrange(3,13))
        

        
# Step 7. 출력 결과 저장하기
newsall = pd.DataFrame()
newsall['언론사']=pd.Series(company2)
newsall['뉴스제목']=pd.Series(title2)
newsall['게시날짜']=pd.Series(date2)
newsall['뉴스내용']=pd.Series(content2)

fx_name = f_dir+'Bigkinds'+'-'+s+'-'+query_txt+'\\'+'Bigkinds'+'-'+query_txt+'.xlsx'

# 엑셀 형태로 저장하기
newsall.to_excel(fx_name,index=True)

print('파일저장경로: %s ' %fx_name)

1.크롤링할 뉴스의 키워드는 무엇입니까?: 코로나19
2.반드시 포함해야 할 단어가 있다면 적어주세요(여러 단어일 경우 , 로 구분해 주세요): 
3.제외할 단어가 있다면 적어주세요:
4.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :2020-01-01
5.조회를 종료할 날짜를 입력하세요(예:2017-12-31): 2020-10-23
6.파일을 저장할 폴더명만 쓰세요(기본값:c:\temp\):c:\temp\


요청하신 데이터를 수집 중입니다


잠시만 기다려 주세요~~~~~^^


Step 1.빅카인즈 홈페이지에 자동 접속 합니다. 잠시만 기다려 주세요 ^^

Step 2.날짜 조건과 검색어를 입력하여 요청하신 뉴스를 검색중입니다. 잠시 후 총 뉴스의 건수가 표시될 예정입니다~^^
Step 3.검색된 기사의 개수는 총 935171 건 입니다
검색된 기사 중에서 크롤링 할 건수를 입력해주세요(최대:935171 건): 30


1 번째 기사 정보를 수집합니다=========================================================
1.언론사명:  매일신문
2.뉴스제목: [기고] 한국산림과학고에서 미래 산림일꾼 키운다
3.게시날짜 : 2020/10/23
4.뉴스내용 : 박문섭 한국산림과학고 산학겸임교사                "숲은 우리의 미래요 희망이다"라는 슬로건에 걸맞은 인재를 육성하기 위해 노력하는 학교가 있다. 바로 국내 유일의 산림계 특성화고인 한국산림과학고등학교이다. 경북 봉화군 춘양면에 위치하고 있는 한국산림과학고는 2012년 춘양상업고등학교에서 교명을 바꿔 새롭게 거듭났으며 현재는 약 140명의 학생들이 산림 전공 분야의 학업에 매진하고 있다.   산림청은 한국산림과학고를 산림 분야 특성화고로 지정하여 2012년부터 지속적으로 '산림 분야 특성화고 지원 사업'을 운영하고 있다. 교육과 현장 간의 연계 강화를 위한 산학겸임교사 채용을 지원하고, 학생들의 현장실무능력 향상과 취·창업 지원을 위한 자격증 취득

1.언론사명:  매일신문
2.뉴스제목: 대구 의용소방대원 120여명 ‘사랑의 헌혈’ 동참
3.게시날짜 : 2020/10/23
4.뉴스내용 : 코로나19로 인한 혈액수급 비상에 자발적 참여                대구시 의용소방대(연합회장 원상연)는 22일 코로나19 장기화로 인한 혈액 부족 사태에 보탬이 되고자 사랑의 헌혈 운동을 펼쳤다.   이날 헌혈 봉사는 대구FC 축구장 주차장에서 오전부터 진행돼 대구시 의용소방대원 120여 명이 참여했다.   대원들은 최근 심해진 혈액 수급난 해소에 기여하고, 혈액을 제때 공급받지 못해 고통받는 이웃들에게 온정을 전하기 위해 자발적으로 동참했다.   헌혈 활동은 감염병 예방을 위해 체온 측정, 마스크 착용, 손 소독 등 개인위생을 강화하고 헌혈버스 소독작업 등 방역지침에 따라 이뤄졌다.   이지만 대구소방안전본부장은 "소방공무원에 이어 의용소방대원들도 헌혈에 자발적으로 참여해줘 감사하다. 앞으로 소방 협력단체 전체로 확대해 헌혈 활동에 지속적으로 동참할 계획이다"고 말했다.


8 번째 기사 정보를 수집합니다=========================================================
1.언론사명:  매일신문
2.뉴스제목: 태전1동 새마을부녀회 ‘마음잇기’ 반찬 나눔
3.게시날짜 : 2020/10/23
4.뉴스내용 : 직접 만든 밑반찬, 관내 50여 독거노인·저소득 가구에 전달                대구 북구 태전1동 새마을부녀회(회장 채부황)는 21일 손수 만든 밑반찬을 저소득층과 독거노인 가구에 나눠주는 '마음잇기' 반찬 나눔 봉사활동을 진행했다.   이날 새마을부녀회 회원들은 어렵고 소외된 이웃에 온정을 나누기 위해 직접 조리한 밑반찬을 관내 50여 가구에 전달했다.   새마을부녀회는 지난 19일에도 직접 만든 추어탕을 소외된 이웃에게 나누며 이웃사랑을 실천한 바 있다. 또한 평소 삼계탕 나눔 봉사, 손소독제 나누기, 가을 꽃 식재 등 밝은 지역사회를 만들기 위해 다양한 활동을 하

1.언론사명:  중도일보
2.뉴스제목: 서산어린이도서관, '도서관과 함께 책읽기' 성료
3.게시날짜 : 2020/10/23
4.뉴스내용 : 서산문화복지센터 어린이도서관(센터장 김금배)은 지역아동센터와 연계하여 운영한 '도서관과 함께 책읽기' 프로그램을 성료했다고 밝혔다. 서산어린이도서관에서 지역아동센터와 연계하여 진행한 '도서관과 함께 책읽기'는 국립어린이청소년도서관이 주최하고, 한국도서관문화진흥원이 주관하며 정보소외계층 어린이들이 올바른 독서습관을 형성하고 정보? 문화적 격차를 해소하기 위한 독서문화프로그램을 지원하는 사업이다. 서산어린이도서관은 이번 공모사업의 운영기관으로 선정되어 강사료, 도서구입비 등의 지원을 받아 도서관과 연계한 지역아동센터에서 초등 1~3학년 12명을 대상으로 운영하였으며, 지난 6월부터 10월 20일까지 총 20회에 걸쳐 독서 전문 강사와 함께하는 독서프로그램 외에도 영화관람, 도서관 활용수업 등 다양한 프로그램으로 수업을 진행해 왔다. 김금배 센터장은 "코로나19로 상황이 어려웠지만 방역에 힘쓰며 문화적 소외를 받는 어린이들이 이번 프로그램을 통해 도서관을 더욱 친근하게 여길 수 있게 되어 기쁘다"며 "앞으로도 도서관 방문이 어려운 정보취약계층을 위해 적극적인 도서관 서비스를 하고자 노력하겠다." 고 하였다. 한편, 서산어린이도서관에서는 오는 31일 과학자들의 재능기부 과학 강연 '10월의 하늘' 강연을 진행할 예정이며,'10월의 하늘' 강연은 초등4학년 이상 청소년부터 성인(90명) 누구나 무료로 참여 가능하며 서산어린이도서관 홈페이지와 이메일을 통해 신청할 수 있다.서산=임붕순 기자 ibs9900@


18 번째 기사 정보를 수집합니다=========================================================
1.언론사명:  중도일보
2.뉴스제목: 코로나19 극복 건강 반찬 꾸러미 나누기 행사 개최
3.게시날짜 : 2020/10/23
4.뉴스내용 : 서산성봉학교,부모 학습공동체 재능기부 실천     

1.언론사명:  KBS
2.뉴스제목: 강원랜드 휴장으로 내년 정선군 세입 430억 원 손실
3.게시날짜 : 2020/10/23
4.뉴스내용 : [KBS 강릉]코로나19로 인한 강원랜드 카지노의 휴장 장기화로, 내년도(2021년) 정선군 세입 손실이 4백억 원이 넘을 것으로 우려됩니다.정선군은 강원랜드 카지노의 장기 휴장에 따라 내년도 폐광기금과 주식배당금은 '0원'으로 예상되고 지방소득세와 상하수도 사용료도 줄어 내년도 세입 손실액이 430억 원으로 예상된다고 밝혔습니다.


27 번째 기사 정보를 수집합니다=========================================================
1.언론사명:  KBS
2.뉴스제목: 강원관광 활성화 기지개…방역수칙 준수 필요
3.게시날짜 : 2020/10/23
4.뉴스내용 : [KBS 강릉][앵커]코로나19로 인한 사회적 거리두기가 1단계로 완화되면서, 강원 동해안을 중심으로 관광산업 활성화 움직임이 재개되고 있습니다.일부 시군은 아직 조심스럽다는 입장인데, 관광 활성화를 위해서라도 방역수칙 준수가 더 중요해졌습니다.강규엽 기자의 보도입니다.[리포트]해발 천4백여 미터 발왕산 정상.제주와 광주, 부산에서 모인 여행사 직원 20여 명이 스카이워크에 직접 올라봅니다.강원도 전담 여행사의 설명을 꼼꼼히 챙겨 듣고, 새로운 여행상품 가능성을 검토합니다.[양원일/강원도 전담 여행사 : "케이블카로 연동이 돼 있고 여기 스카이워크가 설치됐는데, 이게 가장 최근 평창군에 새로 생긴 관광지입니다."]코로나19 사회적 거리두기가 1단계로 완화된 뒤, 강원관광 활성화를 위해 마련된 여행사 직원 팸투어입니다.단체관광 형태가 아닌 가족 단위 힐링관광 위주로 짜여진 관광상품을 홍보했습니다.[양군진/여행사 대표/제주 : "강원도 쪽은 코로나19로부터 상당히 청정지역으로 알려져 있잖아요? 아무래도 우리 제주도민같은 경우도 코로나19에 대한 민감한 반응들이 아무래도 영향이 있어갖고 (강원도에 관심이 있어요)."]강원 동해안 